---
# Install the required packages

If needed install the following packages:

In [1]:
!pip install datasets transformers imbalanced-learn evaluate

---
# Imports

In [28]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DataCollatorWithPadding
from transformers import DistilBertForSequenceClassification
from transformers import TrainingArguments,Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import evaluate
import numpy as np

# Write your code here. Add as many boxes as you need.

---
# Laboratory Exercise - Run Mode (8 points)

## Introduction

This laboratory assignment's primary objective is to fine-tune a pre-trained language model for detection of toxic sentences (binary classification). 

The dataset contains two attributes: 
- `text`: The sentence which needs to be classified in to toxic/non-toxic
- `label`: 0/1 indicator if the given sentence is toxic

**Note: You are required to perform this laboratory assignment on your local machine.**

# Read the data

The dataset reading is given. Just run the following 2 cells.

**DO NOT MODIFY IT! Just analyse how the data reading was performed, as in the future this part won't be given.**

In [3]:
dataset = load_dataset(
    'csv', 
    data_files={'train': '../datasets/train.tsv', 'val': '../datasets/val.tsv','test': '../datasets/test.tsv'},
    delimiter='\t'
)

**The prediction target column MUST be named 'label' in the dataset !**

See the dataset structure:

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 3130
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3132
    })
})

---
# Natural Language Processing

## Generate the Tokenizer and Data Collator

For the purposes of this lab you will be using `DistilBertTokenizer` and `DataCollatorWithPadding`.

In [6]:
# Write your code here. Add as many boxes as you need.
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Tokenize the dataset

For the purposes of lowering the amount of computing set the `max_length` parameter to 15.

In [7]:
# Write your code here. Add as many boxes as you need.
dataset = dataset.map(lambda x: tokenizer(x['text'], padding=True, truncation=True, max_length=15,), batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3130 [00:00<?, ? examples/s]

Map:   0%|          | 0/3132 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3130
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3132
    })
})

## Define the model

The required model for this lab is the `DistilBertForSequenceClassification`.

In [10]:
# Write your code here. Add as many boxes as you need.
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define the training arguments

For lowering the compute time I recommend using the following parameters:
- per_device_train_batch_size=128
- per_device_eval_batch_size=128
- **num_train_epochs=1**

In [15]:
# Write your code here. Add as many boxes as you need.
training_args = TrainingArguments(
    output_dir="./trainer",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    learning_rate = 2e-5
)

## Load the metrics

Load the best metric for the this specific problem.

In [17]:
# Write your code here. Add as many boxes as you need.
metric = evaluate.load("f1")

### Define the function to compute the metrics

In [20]:
# Write your code here. Add as many boxes as you need.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

## Generate the Trainer object

In [31]:
# Write your code here. Add as many boxes as you need.
trainer = Trainer(
    model=model,
    data_collator=collator,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    args=training_args,
    compute_metrics=compute_metrics
)

## Train the model

Use the trainer to train the model.

In [32]:
# Write your code here. Add as many boxes as you need.
trainer.train()

Step,Training Loss


TrainOutput(global_step=8, training_loss=0.6277325749397278, metrics={'train_runtime': 12.3445, 'train_samples_per_second': 81.008, 'train_steps_per_second': 0.648, 'total_flos': 3880880820000.0, 'train_loss': 0.6277325749397278, 'epoch': 1.0})

---
# Evaluate the model

## Generate predictions for the test set

In [33]:
# Write your code here. Add as many boxes as you need.
predictions = trainer.predict(dataset['test'])

## Extract the predictions (class 0 or 1) from the logits

In [34]:
# Write your code here. Add as many boxes as you need.
preds = np.argmax(predictions.predictions, axis=-1)

## Analyze the performance of the model

In [35]:
# Write your code here. Add as many boxes as you need.
print(classification_report(dataset['test']['label'],preds))

              precision    recall  f1-score   support

           0       0.84      0.75      0.79      1566
           1       0.77      0.85      0.81      1566

    accuracy                           0.80      3132
   macro avg       0.81      0.80      0.80      3132
weighted avg       0.81      0.80      0.80      3132



# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify if a given text is **toxic** or not. Use TF-IDF vectorization to convert text into numerical features and train a `MultinomialNB` model. If needed use `RandomUnderSampler()`. Compare the results with the transformer model.

In [16]:
# Write your code here. Add as many boxes as you need.

---